In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard.writer import SummaryWriter

from src import modeling

%load_ext autoreload
%autoreload 2

In [2]:
DATA_DIR = Path('data')
CHECKPOINT_DIR = DATA_DIR / 'models'
CHECKPOINT_DIR.mkdir(exist_ok=True)

In [4]:
IMAGE_SIZE = (128, 128)
dataset_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(IMAGE_SIZE),
    ]
)
dataset_dir = DATA_DIR / "cats_dogs/train"
dataset = torchvision.datasets.ImageFolder(str(dataset_dir), transform=dataset_transform)
labels = dataset.classes

torch.random.manual_seed(69)
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1])

for i in range(0):
    image, label_id = train_dataset[i]
    label = labels[label_id]

    image = transforms.ToPILImage()(image)

    plt.imshow(image)
    plt.title(label)
    plt.axis("off")
    plt.show()

In [5]:
model_path = CHECKPOINT_DIR / 'model.pth'

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

num_channels = 3
num_classes = len(labels)

criterion = nn.CrossEntropyLoss()
model = modeling.Model(
    num_channels=num_channels,
    num_classes=num_classes,
    criterion=criterion,
)

lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

writer = SummaryWriter()
trainer = modeling.Trainer(
    model=model, 
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    writer=writer,
    model_path=model_path,
)

In [ ]:
num_epochs = 6
trainer.train(num_epochs)

In [6]:
model: modeling.Model = torch.load(model_path)
modeling.evaluate_loader(test_loader, model)

Computing metrics:   0%|          | 0/10 [00:00<?, ?it/s]

{'acc': 0.7684000134468079}